# 「分散RPCフレームワーク入門」

【原題】Getting Started with Distributed RPC Framework

【原著】[Shen Li](https://mrshenli.github.io/)

【元URL】https://pytorch.org/tutorials/intermediate/rpc_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年11月28日

【チュトーリアル概要】

前提知識
- [PyTorch Distributedについて](https://pytorch.org/tutorials/beginner/dist_overview.html)（日本語版6_1）
- [RPC APIドキュメント](https://pytorch.org/docs/master/rpc.html)

<br>

本チュートリアルでは、`torch.distributed.rpc` パッケージで分散訓練を構築する方法について、2つのシンプルな例を示しながら解説します。

なお、`torch.distributed.rpc` パッケージは、PyTorch v1.4 から初めてプロトタイプ機能として導入されました。

2つの例について、ソースコードは、[PyTorchのサンプル例](https://github.com/pytorch/examples)で確認できます。


---

以前のチュートリアル（[分散データ並列訓練入門（日本語版6_3）](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)、[PyTorchで実装する分散アプリケーション（日本語版6_4）](https://pytorch.org/tutorials/intermediate/dist_tuto.html)）では、[DistributedDataParallel](https://pytorch.org/docs/stable/_modules/torch/nn/parallel/distributed.html) には、複数のプロセスに渡ってモデルを複製して各プロセスが分割された入力データを扱うといった、特定の訓練パラダイムをサポートするものであると解説しました。



しかし、時には異なる訓練パラダイムが必要になる場面に直面することがあるかもしれません。

例えば以下のようなケースです。

1. 強化学習においては、モデル自体がかなり小さい一方で、環境から得る必要のある訓練データが比較的大量になることがあり得ます。このケースでは、並列に動作する複数のオブザーバーを生成し、単一のエージェントに共有することが有益であるかもしれません。この際、エージェントはローカルで訓練を行いますが、アプリケーションにはオブザーバーとトレーナーの間でデータを送受信するためのライブラリが必要になります。
2. 構築するモデルが単一のマシン上のGPUに収めるには大きすぎるかもしれない場合には、複数のマシンにモデルを分割する上で役に立つライブラリが必要になります。
 そうしない場合、モデルのパラメーターとトレーナーが異なるマシン上に存在する状況を対象にした[パラメータサーバー](https://www.cs.cmu.edu/~muli/file/parameter_server_osdi14.pdf)の訓練フレームワークを実装する必要があるかもしれません。



 [torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) パッケージは、上記のようなシナリオで役に立ちます。

ケース1では、[RPC](https://pytorch.org/docs/master/rpc.html#rpc)と[RRef](https://pytorch.org/docs/master/rpc.html#rref) を使用することで、リモートのデータオブジェクトを簡単に参照しながら、あるワーカーから別のワーカーへデータを送信することができます。

ケース2では、[分散自動微分](https://pytorch.org/docs/master/rpc.html#distributed-autograd-framework)と[分散オプティマイザー](https://pytorch.org/docs/master/rpc.html#module-torch.distributed.optim)を使うことで、あたかもローカル上での訓練のようにバックワードパスとオプティマイザーステップを実行できます。



次の2つのセクションでは、強化学習の例と言語モデルの例を用いて、[torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) のAPIを解説します。

なお、本チュートリアルは、最高の性能や効率のモデルを構築して問題を解くことを目的としておらず、ここでの主な目標は、分散訓練アプリケーションを構築する上で [torch.distributed.rpc](https://pytorch.org/docs/master/rpc.html) を使用する方法を示すことである点に注意してください。

## RPCとRRefを用いた分散強化学習

本セクションでは、[OpenAI Gym](https://gym.openai.com/)のCartPole-v1を対象にRPCを使った、分散強化学習のトイモデル構築手順を解説します。

なお、以下に示すように、ポリシーのコードの大部分は、既存のシングルスレッドの[実装例](https://github.com/pytorch/examples/blob/master/reinforcement_learning)から借用しています。

`Policy`の設計に関する詳細は省き、RPCの使い方に焦点を当てます。

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Policy(nn.Module):

    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

まず、リモートからRRefを保有しているワーカー上で関数を実行する際に役立つ関数を準備します。

この関数は、本チュートリアルの中で何回か目にすることになるでしょう。

本来であれば、`torch.distributed.rpc`がこのような便利な関数を備えているべきです。

例えば、アプリケーションが直接 `RRef.some_func(*arg)` を呼び出すことで、当該関数を、RRefを保有するワーカーに対して処理を行うRPCに変換することができればより楽になります。

このAPIに関する進捗は [pytorch/pytorch#31743](https://github.com/pytorch/pytorch/issues/31743) にて管理されています。

（日本語訳注：2020年1月4日にAPIはマージされ、2020年6月6日にクローズされています。）


In [2]:
from torch.distributed.rpc import rpc_sync

def _call_method(method, rref, *args, **kwargs):
    return method(rref.local_value(), *args, **kwargs)


def _remote_method(method, rref, *args, **kwargs):
    args = [method, rref] + list(args)
    return rpc_sync(rref.owner(), _call_method, args=args, kwargs=kwargs)

# rref上で関数を呼び出すには、次の方法で可能です。
# _remote_method(some_func, rref, *args)

オブザーバーを用意する準備ができました。

今回の例では、各オブザーバーが各自で環境を作成し、エピソードを実行させるエージェントからのコマンドを待機します。

各エピソードでは、一つのオブザーバーが最大で `n_steps` 回のイテレーションをループし、各イテレーションにおいてRPCを用いて環境の状態をエージェントに伝え、アクションを受け取ります。

そして、受け取ったアクションを環境に適用し、環境から報酬と次の状態を得ます。

その後、オブザーバーは、RPCを利用して、エージェントに報酬を報告します。

なお、繰り返しになりますが、この実装方法は、明らかに効率が最大限に良いオブザーバーの実装方法ではない点に留意してください。


例えば、単純な最適化の方法の一つとしては、通信のオーバーヘッドを削減するために現在の状態と最後の報酬を1つのRPCに詰め込むことが考えられます。

しかし、今回の目標はCartPoleを最も上手く攻略することではなく、RPC APIの解説をすることです。

そのため、このチュートリアルではロジックをシンプルにしておき、2つのステップを理解しやすいようにします。

In [ ]:
import argparse
import gym
import torch.distributed.rpc as rpc

parser = argparse.ArgumentParser(
    description="RPC Reinforcement Learning Example",
    formatter_class=argparse.ArgumentDefaultsHelpFormatter,
)

parser.add_argument('--world_size', default=2, help='Number of workers')
parser.add_argument('--log_interval', default=1, help='Log every log_interval episodes')
parser.add_argument('--gamma', default=0.1, help='how much to value future rewards')
parser.add_argument('--seed', default=1, help='random seed for reproducibility')
args = parser.parse_args()

class Observer:

    def __init__(self):
        self.id = rpc.get_worker_info().id
        self.env = gym.make('CartPole-v1')
        self.env.seed(args.seed)

    def run_episode(self, agent_rref, n_steps):
        state, ep_reward = self.env.reset(), 0
        for step in range(n_steps):
            # 状態をエージェントに送信し、アクションを受け取る。
            action = _remote_method(Agent.select_action, agent_rref, self.id, state)

            # アクションを環境に適用し、報酬を受け取る。
            state, reward, done, _ = self.env.step(action)

            # 訓練を行うために、エージェントに報酬を報告する。
            _remote_method(Agent.report_reward, agent_rref, self.id, reward)

            if done:
                break

エージェントのコードはオブザーバーのコードよりも少し複雑なため、複数のパーツに分けて扱います。

今回の例では、エージェントはトレーナーとマスターの両方の役割を果たします。

そのため、エージェントは分散した複数のオブザーバーにコマンドを送信してエピソードを実行させるとともに、各エピソードの後の訓練フェーズで使用されるすべてのアクションと報酬をローカルに記録します。



以下のコードは、`Agent` コンストラクターとなります。

行のほとんどが、各種コンポーネントの初期化に費やされています。

また、コンストラクターの最後のループでは、他のワーカー上のオブザーバーをリモートで初期化し、ローカルでそれらのオブザーバーへのRRefを保持しています。
これらのオブザーバーへのRRefsは、後でエージェントがコマンドを送信するために使用します。

なお、アプリケーションは `RRef` の存続について気にする必要はありません。

`RRef` を保有する各ワーカーは、`RRef` の存続を管理するために参照回数マップを維持し、管理対象の `RRef` を保有しているユーザが存在する限り、リモートのデータオブジェクトが削除されないことを保証しています。

詳細は `RRef` の [設計ドキュメント](https://pytorch.org/docs/master/notes/rref.html) を参照してください。

In [ ]:
import gym
import numpy as np

import torch
import torch.distributed.rpc as rpc
import torch.optim as optim
from torch.distributed.rpc import RRef, rpc_async, remote
from torch.distributions import Categorical

class Agent:
    def __init__(self, world_size):
        self.ob_rrefs = []
        self.agent_rref = RRef(self)
        self.rewards = {}
        self.saved_log_probs = {}
        self.policy = Policy()
        self.optimizer = optim.Adam(self.policy.parameters(), lr=1e-2)
        self.eps = np.finfo(np.float32).eps.item()
        self.running_reward = 0
        self.reward_threshold = gym.make('CartPole-v1').spec.reward_threshold
        for ob_rank in range(1, world_size):
            ob_info = rpc.get_worker_info(OBSERVER_NAME.format(ob_rank))
            self.ob_rrefs.append(remote(ob_info, Observer))
            self.rewards[ob_info.id] = []
            self.saved_log_probs[ob_info.id] = []

次に、アクションの選択と報酬の報告を行うために、エージェントは2つのAPIをオブザーバーに向けて公開します。<br>
これらの関数は、エージェント上でローカルに実行されますが、RPCを介したオブザーバーによるトリガーによって実行されます。

In [ ]:
class Agent:
    # ...
    def select_action(self, ob_id, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.policy(state)
        m = Categorical(probs)
        action = m.sample()
        self.saved_log_probs[ob_id].append(m.log_prob(action))
        return action.item()

    def report_reward(self, ob_id, reward):
        self.rewards[ob_id].append(reward)

エピソードを実行するようにオブザーバーに伝える `run_episode` 関数をエージェントに加えてみましょう。

この関数では、まず非同期RPCから future を収集するためのリストを作成し、その後、すべてのオブザーバーのRRefをループして非同期RPCを作成します。

なお、これらのRPCでは、オブザーバーがエージェント上の関数を呼び出すことができるのと同様に、エージェントも自身のRRefをオブザーバーに渡します。

上に示しましたが、各オブザーバーはエージェントに `RPC` を返しますが、これはネストされた `RPC `です。

各エピソードの後、`saved_log_probs` と `rewards` には、記録されたアクションの確率値と報酬が格納されます。

In [ ]:
class Agent:
    ...
    def run_episode(self, n_steps=0):
        futs = []
        for ob_rref in self.ob_rrefs:
            # 非同期RPCを作り、すべてのオブザーバー上のエピソードを開始する。
            futs.append(
                rpc_async(
                    ob_rref.owner(),
                    _call_method,
                    args=(Observer.run_episode, ob_rref, self.agent_rref, n_steps)
                )
            )

        # すべてのオブザーバーがエピソードを終了するまで待機する。
        for fut in futs:
            fut.wait()

最後の部分です。
1エピソード後、エージェントはモデルを訓練する必要がありますが、この部分については以下の `finish_episode` 関数で実装します。

なお、この関数に `RPC` は存在せず、ほとんどがシングルスレッドの例から借用しています。

そのため、内容の説明は省略します。

In [ ]:
class Agent:
    # ...
    def finish_episode(self):
        # 異なるオブザーバーからの確率値と報酬をリストに結合します。
        R, probs, rewards = 0, [], []
        for ob_id in self.rewards:
            probs.extend(self.saved_log_probs[ob_id])
            rewards.extend(self.rewards[ob_id])

        # 最小のオブザーバーの報酬を使用して実行報酬を計算します。
        min_reward = min([sum(self.rewards[ob_id]) for ob_id in self.rewards])
        self.running_reward = 0.05 * min_reward + (1 - 0.05) * self.running_reward

        # 保存された確率値と報酬を消去します。
        for ob_id in self.rewards:
            self.rewards[ob_id] = []
            self.saved_log_probs[ob_id] = []

        policy_loss, returns = [], []
        for r in rewards[::-1]:
            R = r + args.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + self.eps)
        for log_prob, R in zip(probs, returns):
            policy_loss.append(-log_prob * R)
        self.optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()
        return min_reward

`Policy` クラス、`Observer` クラス、そして `Agent` クラスを用意し、マルチプロセスを立ち上げて分散訓練を行う準備ができました。

今回の例では、すべてのプロセスが同じ `run_worker` 関数を実行しますが、各プロセスの役割を区別するためにはランクを使用します。
具体的には、ランク0は常にエージェントとし、他のランクはすべてオブザーバーとします。

エージェントは、実行中の報酬が環境によって指定された報酬の閾値を超えるまで、`run_episode` と `finish_episode` を繰り返し呼び出すことで、マスターとしての役割を果たします。

一方、すべてのオブザーバーは、エージェントからの命令を受動的に待ち続けます。

なお、コードは [rpc.init_rpc](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.init_rpc) と [rpc.shutdown](https://pytorch.org/docs/master/rpc.html#torch.distributed.rpc.shutdown) によってラップされており、それぞれ RPC インスタンスの初期化と終了を行います。

詳細は、[APIのページ](https://pytorch.org/docs/master/rpc.html) で確認できます。

In [ ]:
import os
from itertools import count

import torch.multiprocessing as mp

AGENT_NAME = "agent"
OBSERVER_NAME="obs"
TOTAL_EPISODE_STEP = 100

def run_worker(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    if rank == 0:
        # rank0はエージェントです。
        rpc.init_rpc(AGENT_NAME, rank=rank, world_size=world_size)

        agent = Agent(world_size)
        for i_episode in count(1):
            n_steps = int(TOTAL_EPISODE_STEP / (args.world_size - 1))
            agent.run_episode(n_steps=n_steps)
            last_reward = agent.finish_episode()

            if i_episode % args.log_interval == 0:
                print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                      i_episode, last_reward, agent.running_reward))

            if agent.running_reward > agent.reward_threshold:
                print("Solved! Running reward is now {}!".format(agent.running_reward))
                break
    else:
        # それ以外のランクはオブザーバーです。
        rpc.init_rpc(OBSERVER_NAME.format(rank), rank=rank, world_size=world_size)
        # オブザーバーは、エージェントからの指令を受動的に待ち続けます。

    # すべてのRPCが終了するまでブロックし、その後RPCインスタンスをシャットダウンします。
    rpc.shutdown()


mp.spawn(
    run_worker,
    args=(args.world_size, ),
    nprocs=args.world_size,
    join=True
)

world_size=2 で訓練した際の出力例の一部は以下の通りです。

```
Episode 10      Last reward: 26.00      Average reward: 10.01
Episode 20      Last reward: 16.00      Average reward: 11.27
Episode 30      Last reward: 49.00      Average reward: 18.62
Episode 40      Last reward: 45.00      Average reward: 26.09
Episode 50      Last reward: 44.00      Average reward: 30.03
Episode 60      Last reward: 111.00     Average reward: 42.23
Episode 70      Last reward: 131.00     Average reward: 70.11
Episode 80      Last reward: 87.00      Average reward: 76.51
Episode 90      Last reward: 86.00      Average reward: 95.93
Episode 100     Last reward: 13.00      Average reward: 123.93
Episode 110     Last reward: 33.00      Average reward: 91.39
Episode 120     Last reward: 73.00      Average reward: 76.38
Episode 130     Last reward: 137.00     Average reward: 88.08
Episode 140     Last reward: 89.00      Average reward: 104.96
Episode 150     Last reward: 97.00      Average reward: 98.74
Episode 160     Last reward: 150.00     Average reward: 100.87
Episode 170     Last reward: 126.00     Average reward: 104.38
Episode 180     Last reward: 500.00     Average reward: 213.74
Episode 190     Last reward: 322.00     Average reward: 300.22
Episode 200     Last reward: 165.00     Average reward: 272.71
Episode 210     Last reward: 168.00     Average reward: 233.11
Episode 220     Last reward: 184.00     Average reward: 195.02
Episode 230     Last reward: 284.00     Average reward: 208.32
Episode 240     Last reward: 395.00     Average reward: 247.37
Episode 250     Last reward: 500.00     Average reward: 335.42
Episode 260     Last reward: 500.00     Average reward: 386.30
Episode 270     Last reward: 500.00     Average reward: 405.29
Episode 280     Last reward: 500.00     Average reward: 443.29
Episode 290     Last reward: 500.00     Average reward: 464.65
Solved! Running reward is now 475.3163778435275!
```

今回の例では、ワーカー間でデータを渡すための通信伝達手段として RPC を使用する方法と、リモートのオブジェクトを参照するために RRef を使用する方法を紹介しました。

その他には、`ProcessGroup` の `send` APIと `recv` API上に構造全体を直接構築したり、他の通信/RPCライブラリを使用することも可能です。

しかし、`torch.distributed.rpc` を使用することで、ネイティブサポートと継続的に最適化されたパフォーマンスを自動的に活用できます。

次のセクションでは、RPCとRRefを分散自動微分と分散オプティマイザーに組み込み、分散モデル並列訓練を実施する方法を解説します。

## 分散自動微分と分散オプティマイザーを用いた分散RNN

本セクションでは、RNNモデルを用いて、分散モデル並列訓練をRPC APIで行う方法について説明します。

なお、今回の例で使用するRNNモデルはとても小さく、単一のGPUにも容易に収まりますが、考え方を解説する目的で2つの異なるワーカーに層を分割します。

また、同様のテクニックを適用することで、開発者は複数のデバイスやマシンに対して、大規模なモデルを分散させることができます。

RNNモデルの設計は、PyTorchの [サンプル例](https://github.com/pytorch/examples/tree/master/word_language_model) のリポジトリにある、単語の言語モデルから借用します。


このモデルには、埋め込みテーブル、LSTM層、デコーダーの3つの主要なコンポーネントが含まれています。



以下のコードは、埋め込みテーブルとデコーダーを1つのサブモジュールにラップし、それらのコンストラクターをRPC APIに渡すようにしています。

なお、`EmbeddingTable` サブモジュールでは、ユースケースを網羅するために、意図的に `Embedding` 層をGPU上に配置しています。

v1.4では、RPCは常にCPUのテンソルの引数や戻り値を宛先のワーカー上に作成します。

そのため、関数がGPU上のテンソルを扱う場合、明示的に適切なデバイスにテンソルを移動させる必要があります。

In [ ]:
class EmbeddingTable(nn.Module):
    """
    RNNModelのエンコード層
    """
    def __init__(self, ntoken, ninp, dropout):
        super(EmbeddingTable, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp).cuda()
        self.encoder.weight.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        return self.drop(self.encoder(input.cuda()).cpu()


class Decoder(nn.Module):
    def __init__(self, ntoken, nhid, dropout):
        super(Decoder, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-0.1, 0.1)

    def forward(self, output):
        return self.decoder(self.drop(output))

上記の2つのサブモジュールにより、RPCを使ってRNNモデルを作成することができるようになりました。

以下の実装コードでは、`ps` が埋め込みテーブルとデコーダーのパラメーターを保有するパラメーターサーバーを表しています。

コンストラクターはリモートAPIを使用し、パラメーターサーバー上で `EmbeddingTable` オブジェクトと `Decoder` オブジェクトを作成した後、`LSTM` サブモジュールをローカルに作成します。

フォワードパスの間に、トレーナーは `EmbeddingTable` のRRefを使ってリモートのサブモジュールを見つけ、RPCで入力データを `EmbeddingTable` に渡し、その後、照会結果を受け取ります。

次に、ローカルの `LSTM` 層を介して埋め込みを実行し、最後に別のRPCを使用して `Decoder` サブモジュールに出力を送信します。


なお一般的に、分散モデル並列訓練を実装する際に開発者は、モデルをサブモジュールに分割し、RPCを呼び出してリモートにサブモジュールインスタンスを作成することで、必要に応じてRRefを使用してそれらのサブモジュールインスタンスを見つけることが可能です。

<br>

下記の実装コードを見てわかるように、シングルマシンモデル並列訓練にとても似たコードになっています。

シングルマシンモデル並列訓練との主な違いは、`Tensor.to(device)` をRPCの関数に置き換えている点です。

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, ps, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()

        # リモートで埋め込みテーブルのセットアップ
        self.emb_table_rref = rpc.remote(ps, EmbeddingTable, args=(ntoken, ninp, dropout))
        # ローカルでLSTMのセットアップ
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        # リモートでデコーダーのセットアップ
        self.decoder_rref = rpc.remote(ps, Decoder, args=(ntoken, nhid, dropout))

    def forward(self, input, hidden):
        # 入力をリモートの埋め込みテーブルに渡し、埋め込みテンソルを受け取る
        emb = _remote_method(EmbeddingTable.forward, self.emb_table_rref, input)
        output, hidden = self.rnn(emb, hidden)
        # 出力をリモートのデコーダーに渡し、デコードされた出力を受け取る
        decoded = _remote_method(Decoder.forward, self.decoder_rref, output)
        return decoded, hidden

分散オプティマイザーについて紹介をする前に、モデルのパラメーターのRRefの配列を生成する上で役に立つ関数を加え、分散オプティマイザーがその配列を扱えるようにしておきましょう。




ローカルでの訓練では、アプリケーションは `Module.parameters()` を呼び出すことですべてのパラメーターのテンソルを照会でき、パラメーター更新のために、それらのテンソルをローカルのオプティマイザーに渡していました。

しかし、一部のパラメーターがリモートのマシン上に存続するため、同じAPIでは分散訓練のケースでは機能しません。

そのため、パラメーターのテンソルの配列を扱う代わりに、分散オプティマイザーはRRefの配列を扱います。

なお、ローカルとリモート双方のモデルのパラメーターについて、モデルのパラメーター毎に一つのRRefを用意します。

作成する関数はいたって単純です。

`Module.parameters()` を呼び出し、パラメーターごとにローカルのRRefを作成するだけです。

In [ ]:
def _parameter_rrefs(module):
    param_rrefs = []
    for param in module.parameters():
        param_rrefs.append(RRef(param))
    return param_rrefs

そして、`RNNModel` は3つのサブモジュールを含んでいるため、`_parameter_rrefs` を3回呼び出す必要があります。
この処理を別の関数にラップします。

In [ ]:
class RNNModel(nn.Module):
    # ...
    def parameter_rrefs(self):
        remote_params = []
        # 埋め込みテーブルのRRefを取得
        remote_params.extend(_remote_method(_parameter_rrefs, self.emb_table_rref))
        # ローカルのパラメーターのRRefを作成
        remote_params.extend(_parameter_rrefs(self.rnn))
        # デコーダーのRRefを取得
        remote_params.extend(_remote_method(_parameter_rrefs, self.decoder_rref))
        return remote_params

以上で、訓練ループを実装する準備ができました。

モデルの引数を初期化した後に、`RNNModel` と `DistributedOptimizer` を作成します。

分散オプティマイザーはパラメーターのRRefの配列を引数に取り、RRefを保有しているすべてのワーカーを見つけます。

そして、与えられた引数（例：`lr=0.05`）を用いて、RRefを保有している各ワーカーにローカルオプティマイザー を作成します(今回のケースでは `SGD` ですが、他のオプティマイザーも使用可能)。

訓練ループでは、まず分散自動微分のコンテクストを作成し、分散自動微分エンジンが勾配、そして関連するRPCである `send` / `recv`関数を見つけられるようにします。

なお、分散自動微分エンジンの詳細な設計については、[設計について](https://pytorch.org/docs/master/notes/distributed_autograd.html)で確認できます。



そして、ローカルモデルのようにフォワードパスに取り掛かり、分散バックワードパスを実行します。

分散バックワードでは、対象となる配列を指定するだけです。今回の場合は、損失の `Tensor` です。

分散自動微分エンジンは、分散された計算グラフを自動的に横断し、各ノードに勾配を適切に書き込みます。

次に、分散オプティマイザー上で `step` 関数を実行することで、すべての関連するローカルオプティマイザーに到達し、モデルのパラメーターを更新します。



なお、ローカルでの訓練と比較して異なる細かい点としては、 `zero_grad()` を実行する必要がない点です。

これは、自動微分の各コンテクストが、勾配を格納するための専用の領域を有していますが、このコンテクストはイテレーションごとに作り直されるため、異なるイテレーションからの勾配が同じ `Tensors` のセットに蓄積されることがないためです。

In [ ]:
def run_trainer():
    batch = 5
    ntoken = 10
    ninp = 2

    nhid = 3
    nindices = 3
    nlayers = 4
    hidden = (
        torch.randn(nlayers, nindices, nhid),
        torch.randn(nlayers, nindices, nhid)
    )

    model = rnn.RNNModel('ps', ntoken, ninp, nhid, nlayers)

    # 分散オプティマイザーのセットアップ
    opt = DistributedOptimizer(
        optim.SGD,
        model.parameter_rrefs(),
        lr=0.05,
    )

    criterion = torch.nn.CrossEntropyLoss()

    def get_next_batch():
        for _ in range(5):
            data = torch.LongTensor(batch, nindices) % ntoken
            target = torch.LongTensor(batch, ntoken) % nindices
            yield data, target

    # 10イテレーションの訓練
    for epoch in range(10):
        for data, target in get_next_batch():
            # 分散自動微分のコンテクストを作成
            with dist_autograd.context() as context_id:
                hidden[0].detach_()
                hidden[1].detach_()
                output, hidden = model(data, hidden)
                loss = criterion(output, target)
                # 分散バックワードパスの実行
                dist_autograd.backward(context_id, [loss])
                # 分散オプティマイザーの実行
                opt.step(context_id)
                # 勾配は、毎イテレーションでリセットされる分散自動微分に
                # 累積されていくため、
                # 勾配をゼロ化する必要はありません。
        print("Training epoch {}".format(epoch))

最後に、パラメータサーバーと訓練プロセスを起動するためのグルーコード（2つのモジュールを接着する補助関数）を追加しましょう。

In [ ]:
def run_worker(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    if rank == 1:
        rpc.init_rpc("trainer", rank=rank, world_size=world_size)
        _run_trainer()
    else:
        rpc.init_rpc("ps", rank=rank, world_size=world_size)
        # パラメーターサーバーは特に何も行いません。
        pass

    # すべてのrpcが終了するまで、処理をブロックします。
    rpc.shutdown()


if __name__=="__main__":
    world_size = 2
    mp.spawn(run_worker, args=(world_size, ), nprocs=world_size, join=True)